In [1]:
import kaggle

! kaggle competitions download -c tabular-playground-series-sep-2022


  0%|          | 0.00/615k [00:00<?, ?B/s]
100%|##########| 615k/615k [00:00<00:00, 5.04MB/s]
100%|##########| 615k/615k [00:00<00:00, 5.04MB/s]


# IMPORTING LIBRARIES AND DATA

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [4]:
df_train=pd.read_csv("C:/Users/nagul/Desktop/Python Work/tabular-playground-series-sep-2022/train.csv")
df_test=pd.read_csv("C:/Users/nagul/Desktop/Python Work/tabular-playground-series-sep-2022/test.csv")

# SOME EXPLORATORY ANALYSIS

In [39]:
df_train.tail()

,row_id,date,country,store,product,num_sold
70123,70123,2020-12-31,Spain,KaggleMart,Kaggle for Kids: One Smart Goose,614
70124,70124,2020-12-31,Spain,KaggleRama,Kaggle Advanced Techniques,215
70125,70125,2020-12-31,Spain,KaggleRama,Kaggle Getting Started,158
70126,70126,2020-12-31,Spain,KaggleRama,Kaggle Recipe Book,135
70127,70127,2020-12-31,Spain,KaggleRama,Kaggle for Kids: One Smart Goose,202


In [10]:
print("Original training set has this many rows:", df_train.shape[0])
print("Original training set has this many columns:", df_train.shape[1])

Original training set has this many rows: 70128
Original training set has this many columns: 6


In [17]:
df_train.describe(include="all")

,row_id,date,country,store,product,num_sold
count,70128.000000,70128,70128,70128,70128,70128.000000
unique,NaN,1461,6,2,4,NaN
top,NaN,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,NaN
freq,NaN,48,11688,35064,17532,NaN
mean,35063.500000,NaN,NaN,NaN,NaN,194.296986
std,20244.354176,NaN,NaN,NaN,NaN,126.893874
min,0.000000,NaN,NaN,NaN,NaN,19.000000
25%,17531.750000,NaN,NaN,NaN,NaN,95.000000
50%,35063.500000,NaN,NaN,NaN,NaN,148.000000
75%,52595.250000,NaN,NaN,NaN,NaN,283.000000


In [76]:
print(df_train["product"].value_counts())
print(df_train["country"].value_counts())
print(df_train["store"].value_counts())

Kaggle Advanced Techniques          17532
Kaggle Getting Started              17532
Kaggle Recipe Book                  17532
Kaggle for Kids: One Smart Goose    17532
Name: product, dtype: int64
Belgium    11688
France     11688
Germany    11688
Italy      11688
Poland     11688
Spain      11688
Name: country, dtype: int64
KaggleMart    35064
KaggleRama    35064
Name: store, dtype: int64


In [24]:
print(df_train.isna().sum())
print("\n",df_train.dtypes)

row_id      0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

 row_id       int64
date        object
country     object
store       object
product     object
num_sold     int64
dtype: object


In [38]:
df_train[df_train.isna().any(1)]

,row_id,date,country,store,product,num_sold


In [42]:
df_train["date"] = pd.to_datetime(df_train["date"],format="%Y-%m-%d")
df_train.dtypes

row_id               int64
date        datetime64[ns]
country             object
store               object
product             object
num_sold             int64
dtype: object

In [43]:
df_train["dayofweek"]=df_train["date"].dt.day_name()
df_train["month"]=df_train["date"].dt.month_name()

df_train.head()

,row_id,date,country,store,product,num_sold,dayofweek,month
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,Sunday,January
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615,Sunday,January
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480,Sunday,January
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,Sunday,January
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240,Sunday,January


In [45]:
modified_training_df=df_train.drop(columns=["row_id","date"],axis=1).copy()
modified_training_df.head()

,country,store,product,num_sold,dayofweek,month
0,Belgium,KaggleMart,Kaggle Advanced Techniques,663,Sunday,January
1,Belgium,KaggleMart,Kaggle Getting Started,615,Sunday,January
2,Belgium,KaggleMart,Kaggle Recipe Book,480,Sunday,January
3,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,Sunday,January
4,Belgium,KaggleRama,Kaggle Advanced Techniques,240,Sunday,January


In [51]:

X_encoded=pd.get_dummies(modified_training_df,
                         columns=["country","store","product",
                                  "dayofweek","month"])
y=X_encoded["num_sold"]
X_encoded=X_encoded.drop(columns=["num_sold"],axis=1)
X_encoded.head()

,country_Belgium,country_France,country_Germany,country_Italy,country_Poland,country_Spain,store_KaggleMart,store_KaggleRama,product_Kaggle Advanced Techniques,product_Kaggle Getting Started,...,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September
0,1,0,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0


In [92]:
for columns in X_encoded:
    #print(X_encoded.columns.value_counts())
    print(X_encoded[columns].value_counts())

0    58440
1    11688
Name: country_Belgium, dtype: int64
0    58440
1    11688
Name: country_France, dtype: int64
0    58440
1    11688
Name: country_Germany, dtype: int64
0    58440
1    11688
Name: country_Italy, dtype: int64
0    58440
1    11688
Name: country_Poland, dtype: int64
0    58440
1    11688
Name: country_Spain, dtype: int64
1    35064
0    35064
Name: store_KaggleMart, dtype: int64
0    35064
1    35064
Name: store_KaggleRama, dtype: int64
0    52596
1    17532
Name: product_Kaggle Advanced Techniques, dtype: int64
0    52596
1    17532
Name: product_Kaggle Getting Started, dtype: int64
0    52596
1    17532
Name: product_Kaggle Recipe Book, dtype: int64
0    52596
1    17532
Name: product_Kaggle for Kids: One Smart Goose, dtype: int64
0    60144
1     9984
Name: dayofweek_Friday, dtype: int64
0    60096
1    10032
Name: dayofweek_Monday, dtype: int64
0    60144
1     9984
Name: dayofweek_Saturday, dtype: int64
0    60096
1    10032
Name: dayofweek_Sunday, dtype: int64


In [97]:
X_train,X_test,y_train,y_test=train_test_split(X_encoded,y,random_state=42)

In [98]:
X_train

,country_Belgium,country_France,country_Germany,country_Italy,country_Poland,country_Spain,store_KaggleMart,store_KaggleRama,product_Kaggle Advanced Techniques,product_Kaggle Getting Started,...,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September
54452,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
54669,0,0,0,0,0,1,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
62222,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
69056,0,0,0,0,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
12792,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,0,0,0,0,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6265,0,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
54886,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
860,0,0,0,0,0,1,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0


[663,
 615,
 480,
 710,
 240,
 187,
 158,
 267,
 610,
 463,
 407,
 672,
 220,
 172,
 140,
 255,
 700,
 562,
 460,
 826,
 240,
 204,
 163,
 282,
 497,
 404,
 346,
 534,
 190,
 149,
 124,
 187,
 222,
 186,
 146,
 236,
 79,
 63,
 50,
 92,
 447,
 364,
 313,
 451,
 159,
 123,
 113,
 181,
 514,
 408,
 342,
 601,
 187,
 158,
 119,
 196,
 455,
 364,
 286,
 506,
 159,
 131,
 101,
 180,
 537,
 410,
 355,
 594,
 185,
 145,
 134,
 214,
 391,
 330,
 245,
 409,
 135,
 101,
 85,
 152,
 161,
 135,
 102,
 199,
 55,
 45,
 40,
 67,
 339,
 266,
 236,
 379,
 124,
 104,
 74,
 123,
 549,
 425,
 334,
 515,
 172,
 131,
 120,
 188,
 465,
 362,
 252,
 433,
 155,
 114,
 101,
 161,
 510,
 415,
 338,
 558,
 193,
 127,
 119,
 191,
 336,
 268,
 245,
 361,
 128,
 95,
 91,
 127,
 172,
 121,
 107,
 171,
 60,
 47,
 35,
 61,
 320,
 271,
 211,
 335,
 113,
 87,
 69,
 125,
 477,
 384,
 328,
 517,
 177,
 134,
 115,
 169,
 465,
 311,
 281,
 460,
 152,
 119,
 102,
 163,
 417,
 403,
 327,
 524,
 181,
 133,
 114,
 182,
 337,
 279